# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('cities.csv')
cities.head()
cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Khatanga,1,RU,1597435367,57,71.97,102.50,49.55,15.59
1,1,Castro,42,BR,1597435221,75,-24.79,-50.01,69.58,5.10
2,2,Ribeira Grande,40,PT,1597435370,83,38.52,-28.70,75.20,11.41
3,3,Bachaquero,53,VE,1597435371,61,9.97,-71.15,89.24,8.70
4,4,Hasaki,20,JP,1597435305,100,35.73,140.83,78.80,3.36
...,...,...,...,...,...,...,...,...,...,...
573,573,Teluknaga,40,ID,1597436066,75,-6.10,106.64,78.01,5.82
574,574,Sheridan,1,US,1597436068,16,44.83,-106.92,80.01,10.29
575,575,Luau,0,AO,1597436069,24,-10.71,22.22,68.29,3.38
576,576,Menongue,0,AO,1597436070,30,-14.66,17.69,58.57,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()
locats = cities[['Lat', 'Lng']]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locats, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
poss_cities_df = cities.loc[(cities['Max Temp'] > 70) & (cities['Max Temp'] < 80) & (cities['Cloudiness'] == 0), :]
poss_cities_df = poss_cities_df.dropna(how='any')
poss_cities_df.reset_index(inplace=True)
del poss_cities_df['index']
hotel_df = poss_cities_df
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,24,Makat,0,KZ,1597435397,41,47.65,53.35,73.87,8.48
1,119,Caravelas,0,BR,1597435512,76,-17.71,-39.25,74.30,6.64
2,155,Magnor,0,SE,1597435555,64,59.95,12.20,71.60,5.82
3,222,Auki,0,SB,1597435636,82,-8.77,160.70,77.52,2.21
4,241,Iclod,0,RO,1597435658,56,46.98,23.80,75.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotellist = []

for i in range(len(hotel_df)):
    
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    json = request.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append('')
hotel_df['Hotel Name'] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,24,Makat,0,KZ,1597435397,41,47.65,53.35,73.87,8.48,
1,119,Caravelas,0,BR,1597435512,76,-17.71,-39.25,74.30,6.64,
2,155,Magnor,0,SE,1597435555,64,59.95,12.20,71.60,5.82,
3,222,Auki,0,SB,1597435636,82,-8.77,160.70,77.52,2.21,
4,241,Iclod,0,RO,1597435658,56,46.98,23.80,75.00,2.24,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locats)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))